<a href="https://colab.research.google.com/github/chawlaKat/Notebook-Noodlings/blob/master/Get_Scattered_Fancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[insert intro here]

get_scattered got too long

im trying to make a decent-looking interactive scatter plot from .arff data

In [1]:
pip install liac_arff

  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built liac-arff


In [0]:
import bark_to_byte as bb
import altair as alt
import pandas as pd

In [0]:
algSource = bb.arff_to_dataframe('abstracted_algorithm_runs.arff')

In [0]:
alg_rekeyed = algSource.set_index('instance_id')
alg_joined = alg_rekeyed.join(alg_rekeyed, lsuffix='_x', rsuffix='_y')
alg_joined.reset_index(inplace=True)

In [0]:
four_colors = ['#ffe200', '#f5ffff', '#6600db', '#11182b']
three_colors = ['#dd1f75', '#fda900', '#2a4dac']
three_colors_alt = ['#6754ad', '#ffffff', '#2d5d00']

In [0]:
def make_graph(data_source, x_axis, y_axis, point_marker = 'circle'):
  
  if(point_marker == 'circle'):
      new_graph = alt.Chart(data_source).mark_circle(size=60).encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'rect'):
      new_graph = alt.Chart(data_source).mark_rect().encode(
          x = x_axis,
          y = y_axis
      )
      
  elif(point_marker == 'bar'):
      new_graph = alt.Chart(data_source).mark_bar().encode(
          x = x_axis,
          y = y_axis
      )
    
  else:
      new_graph = alt.Chart(data_source).mark_point().encode(
          x = x_axis,
          y = y_axis
      )
  
  return new_graph

In [0]:
def make_dropdown(data_source, column, name_prefix):
  opts = list(data_source[column].unique())
  dropdown = alt.binding_select(options = opts)
  select = alt.selection_single(fields = [column],
                                bind = dropdown,
                                name = name_prefix
                               )
  
  return select

In [0]:
def make_selection(array_of_fields, is_multi = False):
  if (is_multi):
    selection = alt.selection_multi(fields = array_of_fields)
  else:
    selection = alt.selection_single(fields = array_of_fields)
    
  return selection

In [0]:
def make_color_scheme(data_source, column, colors, data_type_encoding = 'N'):
  scheme_domain = list(data_source[column].unique())
  scheme_range = colors
  
  scheme_scale = alt.Scale(domain = scheme_domain,
                    range = scheme_range)
  
  #altair needs to know if data is nominal, quantitative, ordinal, or temporal
  if(data_type_encoding == 'N' or data_type_encoding == 'Q' or
      data_type_encoding == 'O' or data_type_encoding == 'T'):
    column_typed = column + ':' + data_type_encoding
  else:
    column_typed = column + ':N'
  
  color_scheme = alt.Color(column_typed,
                           legend = None,
                           scale = scheme_scale)
  
  return color_scheme

In [0]:
def make_color_condition(selector, color_true, color_false = alt.value('darkgray')):
  condition = alt.condition(selector,
                            color_true,
                            color_false)
  
  return condition

In [0]:
def apply_selection_color(chart, selector, color_condition):
  colored_chart = chart.add_selection(
        selector
      ).encode(
        color = color_condition
      )
  
  return colored_chart

In [0]:
def create_comparative_scatter(data_source, key, x_category, y_category, 
                               x_data, y_data, x_filter, y_filter,
                               data_color):
  
  #filter data on x and y axis
  data_legend_base = make_graph(data_source, x_category, y_category, 'rect')
  data_legend_selector = make_selection([x_category, y_category])
  data_legend_colors = make_color_scheme(data_source, y_category, data_color)
  data_legend_update = make_color_condition(data_legend_selector, 
                                            data_legend_colors)
  data_legend = apply_selection_color(data_legend_base, data_legend_selector,
                                      data_legend_update)
  
  #filter data based on attribute
  filter_legend_base = make_graph(data_source, x_filter, y_filter, 'rect')
  filter_legend_selector = make_selection([x_filter, y_filter], True)
  filter_legend_colors = make_color_scheme(data_source, y_filter, data_color)
  filter_legend_update = make_color_condition(filter_legend_selector,
                                              filter_legend_colors)
  filter_legend = apply_selection_color(filter_legend_base, 
                                        filter_legend_selector, 
                                        filter_legend_update)
  
  #set up scatter plot to listen to both legends
  scatter_base = make_graph(data_source, x_data, y_data)
  scatter_tooltip = scatter_base.encode(
                                tooltip = [key, x_category, x_data, x_filter,
                                           y_category, y_data, y_filter]
                            )
  scatter_plot = scatter_tooltip.transform_filter(data_legend_selector
                            ).transform_filter(filter_legend_selector
                            ).interactive()
  
  
  comparative_chart = (data_legend & filter_legend) | scatter_plot
  
  return comparative_chart

In [0]:
algChart = make_graph(algSource, 'algorithm', 'sum(runtime)', 'bar')

algChart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
instance_dropdown = make_dropdown(algSource, 'instance_id', 'in_id')

algDropdown = algChart.add_selection(
    instance_dropdown).transform_filter(
    instance_dropdown)

algDropdown

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'in_id': SelectionDef({
    bind: BindRadioSelect({
      input: 'select',
      options: ['Instance_0', 'Instance_1', 'Instance_2', 'Instance_3', 'Instance_4', 'Instance_5']
    }),
    fields: ['instance_id'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'in_id'}
  })]
})

In [0]:
alg_legend = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm_x',
      type: 'nominal'
    }),
    y: Y({
      field: 'algorithm_y',
      type: 'nominal'
    })
  }),
  mark: 'rect'
})

In [0]:
alg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)

alg_legend_colored = alg_legend.encode(
    color = alg_colors
)

alg_legend_colored

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm_y',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    })

In [0]:
alg_color_scheme = make_color_scheme(algSource, 'algorithm', four_colors)

alg_chart_colored = algChart.encode(color = alg_color_scheme)

alg_chart_colored

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: Color({
      field: 'algorithm',
      legend: None,
      scale: Scale({
        domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
        range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar'
})

In [0]:
alg_selection = make_selection(['algorithm'])

with_select = algChart.add_selection(alg_selection).transform_filter(alg_selection)
with_select

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector009': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })},
  transform: [FilterTransform({
    filter: {'selection': 'selector009'}
  })]
})

In [0]:
alg_selector = make_selection(['algorithm'])
alg_color = make_color_scheme(algSource, 'algorithm', four_colors)
alg_color_update = make_color_condition(alg_selector, alg_color)

fresh_chart = algChart.add_selection(alg_selector).encode(color = alg_color_update)

fresh_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector047',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector047': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [0]:
colorable_chart = apply_selection_color(algChart, alg_selector, alg_color_update)
                                        
colorable_chart

Chart({
  data:    instance_id  repetition       algorithm   runtime runstatus
  0   Instance_0         1.0     algorithm_0  3600.000    memout
  1   Instance_0         1.0     algorithm_1  3600.000    memout
  2   Instance_0         1.0  id_algorithm_0   311.863        ok
  3   Instance_0         1.0  id_algorithm_1  3600.000   timeout
  4   Instance_1         1.0     algorithm_0  3600.000    memout
  5   Instance_1         1.0     algorithm_1  3600.000    memout
  6   Instance_1         1.0  id_algorithm_0    78.121        ok
  7   Instance_1         1.0  id_algorithm_1   600.402        ok
  8   Instance_2         1.0     algorithm_0  3600.000    memout
  9   Instance_2         1.0     algorithm_1  3600.000    memout
  10  Instance_2         1.0  id_algorithm_0     0.012        ok
  11  Instance_2         1.0  id_algorithm_1    60.720        ok
  12  Instance_3         1.0     algorithm_0  3600.000    memout
  13  Instance_3         1.0     algorithm_1    92.986        ok
  14  Instance_3         1.0  id_algorithm_0  2031.039        ok
  15  Instance_3         1.0  id_algorithm_1   141.857        ok
  16  Instance_4         1.0     algorithm_0  3600.000    memout
  17  Instance_4         1.0     algorithm_1  3600.000    memout
  18  Instance_4         1.0  id_algorithm_0   422.530        ok
  19  Instance_4         1.0  id_algorithm_1  3600.000   timeout
  20  Instance_5         1.0     algorithm_0   182.019        ok
  21  Instance_5         1.0     algorithm_1  3600.000    memout
  22  Instance_5         1.0  id_algorithm_0    66.988        ok
  23  Instance_5         1.0  id_algorithm_1   249.488        ok,
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db', '#11182b']
        }),
        selection: 'selector047',
        type: StandardType('nominal')
      }),
      value: 'darkgray'
    }),
    x: X({
      field: 'algorithm',
      type: 'nominal'
    }),
    y: Y({
      aggregate: 'sum',
      field: 'runtime',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  selection: {'selector047': SelectionDef({
    fields: ['algorithm'],
    type: 'single'
  })}
})

In [0]:
legend_base = make_graph(alg_joined, 'runstatus_x', 'runstatus_y', 'rect')
legend_select = make_selection(['runstatus_x', 'runstatus_y'], True)
legend_colors = make_color_scheme(alg_joined, 'runstatus_y', three_colors)
legend_color_update = make_color_condition(legend_select, legend_colors, alt.value('darkgray'))
legend = apply_selection_color(legend_base, legend_select, legend_color_update)

legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('runstatus_y'),
        legend: None,
        scale: Scale({
          domain: ['memout', 'ok', 'timeout'],
          range: ['#dd1f75', '#fda900', '#2a4dac']
        }),
        selection: 'selec

In [0]:
a_leg_base = make_graph(alg_joined, 'algorithm_x', 'algorithm_y', 'rect')
a_leg_select = make_selection(['algorithm_x', 'algorithm_y'])
a_leg_colors = make_color_scheme(alg_joined, 'algorithm_y', four_colors)
a_leg_col_update = make_color_condition(a_leg_select, a_leg_colors)
alg_legend = apply_selection_color(a_leg_base, a_leg_select, a_leg_col_update)

alg_legend

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    color: ColorValue({
      condition: Color({
        field: FieldName('algorithm_y'),
        legend: None,
        scale: Scale({
          domain: ['algorithm_0', 'algorithm_1', 'id_algorithm_0', 'id_algorithm_1'],
          range: ['#ffe200', '#f5ffff', '#6600db'

In [0]:
scatter_base = make_graph(alg_joined, 'runtime_x', 'runtime_y')
scatter_filtered = scatter_base.transform_filter(
                      legend_select
                   ).transform_filter(
                      a_leg_select
                   ).interactive()

(alg_legend & legend) | scatter_filtered

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [26]:
create_comparative_scatter(alg_joined, 'instance_id', 'algorithm_x', 'algorithm_y',
                           'runtime_x', 'runtime_y', 'runstatus_x', 
                           'runstatus_y', four_colors)

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
def save_scatter(chart, name='chart'):
  chart.save(name+'.html')

In [0]:
sample_scatter = create_comparative_scatter(alg_joined, 'algorithm_x', 'algorithm_y', 'runtime_x', 'runtime_y', 
                                            'runstatus_x', 'runstatus_y', four_colors)

In [16]:
sample_scatter

HConcatChart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  hconcat: [VConcatChart({
    vconcat: [Chart({
      encoding: FacetedEncoding({
        color: ColorValue({
          condition: Color({
            field: FieldName('algorithm_y'),
            legend: None,
            scale: Scale({
              domain: ['algorithm_0', 'algorithm_1', 'i

In [0]:
save_scatter(sample_scatter, 'sample scatter')

In [19]:
tester = make_graph(alg_joined, 'runtime_x', 'runtime_y')
tester

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    type: 'circle'
  })
})

In [21]:
test_tip = tester.encode(tooltip = ['instance_id'])
test_tip

Chart({
  data:    instance_id  repetition_x  ... runtime_y  runstatus_y
  0   Instance_0           1.0  ...  3600.000       memout
  1   Instance_0           1.0  ...  3600.000       memout
  2   Instance_0           1.0  ...   311.863           ok
  3   Instance_0           1.0  ...  3600.000      timeout
  4   Instance_0           1.0  ...  3600.000       memout
  5   Instance_0           1.0  ...  3600.000       memout
  6   Instance_0           1.0  ...   311.863           ok
  7   Instance_0           1.0  ...  3600.000      timeout
  8   Instance_0           1.0  ...  3600.000       memout
  9   Instance_0           1.0  ...  3600.000       memout
  10  Instance_0           1.0  ...   311.863           ok
  11  Instance_0           1.0  ...  3600.000      timeout
  12  Instance_0           1.0  ...  3600.000       memout
  13  Instance_0           1.0  ...  3600.000       memout
  14  Instance_0           1.0  ...   311.863           ok
  15  Instance_0           1.0  ...  3600.000      timeout
  16  Instance_1           1.0  ...  3600.000       memout
  17  Instance_1           1.0  ...  3600.000       memout
  18  Instance_1           1.0  ...    78.121           ok
  19  Instance_1           1.0  ...   600.402           ok
  20  Instance_1           1.0  ...  3600.000       memout
  21  Instance_1           1.0  ...  3600.000       memout
  22  Instance_1           1.0  ...    78.121           ok
  23  Instance_1           1.0  ...   600.402           ok
  24  Instance_1           1.0  ...  3600.000       memout
  25  Instance_1           1.0  ...  3600.000       memout
  26  Instance_1           1.0  ...    78.121           ok
  27  Instance_1           1.0  ...   600.402           ok
  28  Instance_1           1.0  ...  3600.000       memout
  29  Instance_1           1.0  ...  3600.000       memout
  ..         ...           ...  ...       ...          ...
  66  Instance_4           1.0  ...   422.530           ok
  67  Instance_4           1.0  ...  3600.000      timeout
  68  Instance_4           1.0  ...  3600.000       memout
  69  Instance_4           1.0  ...  3600.000       memout
  70  Instance_4           1.0  ...   422.530           ok
  71  Instance_4           1.0  ...  3600.000      timeout
  72  Instance_4           1.0  ...  3600.000       memout
  73  Instance_4           1.0  ...  3600.000       memout
  74  Instance_4           1.0  ...   422.530           ok
  75  Instance_4           1.0  ...  3600.000      timeout
  76  Instance_4           1.0  ...  3600.000       memout
  77  Instance_4           1.0  ...  3600.000       memout
  78  Instance_4           1.0  ...   422.530           ok
  79  Instance_4           1.0  ...  3600.000      timeout
  80  Instance_5           1.0  ...   182.019           ok
  81  Instance_5           1.0  ...  3600.000       memout
  82  Instance_5           1.0  ...    66.988           ok
  83  Instance_5           1.0  ...   249.488           ok
  84  Instance_5           1.0  ...   182.019           ok
  85  Instance_5           1.0  ...  3600.000       memout
  86  Instance_5           1.0  ...    66.988           ok
  87  Instance_5           1.0  ...   249.488           ok
  88  Instance_5           1.0  ...   182.019           ok
  89  Instance_5           1.0  ...  3600.000       memout
  90  Instance_5           1.0  ...    66.988           ok
  91  Instance_5           1.0  ...   249.488           ok
  92  Instance_5           1.0  ...   182.019           ok
  93  Instance_5           1.0  ...  3600.000       memout
  94  Instance_5           1.0  ...    66.988           ok
  95  Instance_5           1.0  ...   249.488           ok
  
  [96 rows x 9 columns],
  encoding: FacetedEncoding({
    tooltip: [Tooltip({
      field: 'instance_id',
      type: 'nominal'
    })],
    x: X({
      field: 'runtime_x',
      type: 'quantitative'
    }),
    y: Y({
      field: 'runtime_y',
      type: 'quantitative'
    })
  }),
  mark: MarkDef({
    size: 60,
    ty